In [ ]:
# 영화 리뷰 긍정, 부정 판단 모델
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
print(tf.__version__)

!pip install -q tensorflow-datasets

2.4.0


In [4]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True) # TFDS(텐서플로우 데이터셋. 텐서플로우에서 제공하는 데이터다.)에서 imdb 데이터셋 불러오기. imdb에는 리뷰 내용과 리뷰의 긍정, 부정에 대한 판단이 들어가고 info는...뭐지? 


:05<00:02, 4394.95 examples/s]
Generating train examples...:  99%|█████████▉| 24813/25000 [00:07<00:00, 4400.06 examples/s]
                                                                                            
Generating test examples...:  99%|█████████▉| 24703/25000 [00:05<00:00, 4841.87 examples/s]
                                                                                           
Generating unsupervised examples...: 100%|█████████▉| 49768/50000 [00:14<00:00, 4724.52 examples/s]
                                                                                                   
Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]
Shuffling imdb_reviews-unsupervised.tfrecord...:  95%|█████████▍| 47395/50000 [00:00<00:00, 104386.69 examples/s]
Dataset imdb_reviews downloaded and prepared to /Users/minguinho/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [6]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test'] # imdb에서 훈련셋과 테스트셋을 불러온다. 따로 저장되어있는듯 

# 훈련셋의 문장(입력값), 라벨(긍정인가 부정인가? 출력값) 담는 리스트 
training_sentences = []
training_labels = []

# 테스트셋의 문장, 라벨 담는 리스트
testing_sentences = []
testing_labels = []

# 훈련셋에서 s(sentences), l(label)을 뽑아낸 뒤 training_sentences, training_labels에 담는다. 
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8')) # 문자열을 리스트에 담아주는거기 때문에 utf8이나 유니코드 등의 인코딩 방식을 선택해야한다. 
  training_labels.append(l.numpy())
  
# 테스트셋
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

# 넘파이 배열로 바꿔준다. 왜냐하면 나중에 훈련시킬 때 넘파이 배열을 받기 때문이다. 
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [7]:
vocab_size = 10000 # 단어 크기
embedding_dim = 16 # 임패딩 단위
max_length = 120 # 리뷰에 들어있는 최대 단어 갯수. 아무리 길어도 단어 120개가 한계다...라는 것을 나타냄.
trunc_type='post' # truncation type. 단어 갯수가 120개보다 많을 때 앞에서부터 자를건지 뒤에서부터 자를건지 결정함 post는 문자열이 120개보다 많으면 뒤에서 자르겠다는 뜻.
oov_tok = "<OOV>" # 토큰 딕셔너리에 없는 단어는 OOV로 표시


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok) # num_words = vocab_size : 가장 많이 쓰이는 단어 'vocab_size'개만 토큰화 하는거. 그러니까 토큰화 대상인 문자여렝 단어가 20000개 있으면 많이 쓰이는 순으로 나열한 뒤 앞에 있는 10000개의 단어만 토큰화 하고 나머지 10000개는 토큰화하지 않는다. 얘들은 뭐 OOV로 나오겠지
tokenizer.fit_on_texts(training_sentences) # 토큰화
word_index = tokenizer.word_index # 토큰화 결과 가져오기
sequences = tokenizer.texts_to_sequences(training_sentences) # 문장들을 토큰 순서로 나열하기
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type) # 패딩. 

testing_sequences = tokenizer.texts_to_sequences(testing_sentences) # 훈련셋으로 만든 토크나이저로 테스트셋의 시퀀스 출력. 토큰화 하지않은 단어가 많기 때문에 기존에 만들어놨던 걸로 한다. 만약 토크나이저를 새로 만든다면 두 토크나이저는 다른 토크나이저가 되기 때문에 원하는 결과를 얻을 수 없다. 
testing_padded = pad_sequences(testing_sequences,maxlen=max_length) # 패딩


In [9]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()]) # 딕셔너리가 원래 [단어 : 토큰]이었는데 이걸 [토큰 : 단어]로 바꾼다. 왜냐하면 이렇게 해야 딕셔너리[i]로 단어를 불러와 시퀀스를 문장으로 만들 수 있거든. 
# 효율적인 작업을 위한 준비라고 보면 되겠다. 

# 시퀀스 -> 문장
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text]) # join : 문자열 합치기. get(i, '?')는 i에 해당하는 value(여기서는 단어)가 없으면 ?를 출력하겠다는 의미다. 

print(decode_review(padded[1]))
print(training_sentences[1])

? ? ? ? ? ? ? ? i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what

In [10]:
# 모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), # 임베딩 레이어. 단어의 의미를 단어의 벡터 방향으로 추측하게 하는 것. 각 단어마다 16개의 차원(측정 요소가 16개)을 가진다. 
    # 두 단어가 가리키는 방향이 비슷하면 두 단어의 의미가 비슷한거다. 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # 이진 분류니까 얘 사용
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) # 컴파일. 긍정, 부정밖에 없으니 이진 분류를 하니까 binary_crossentropy를 손실 함수로 사용한다. 
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [11]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final)) # 훈련

Epoch 1/10
782/782 [==============================] - 3s 2ms/step - loss: 0.6092 - accuracy: 0.6273 - val_loss: 0.3428 - val_accuracy: 0.8490
Epoch 2/10
782/782 [==============================] - 1s 2ms/step - loss: 0.2331 - accuracy: 0.9123 - val_loss: 0.3664 - val_accuracy: 0.8414
Epoch 3/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0773 - accuracy: 0.9852 - val_loss: 0.4537 - val_accuracy: 0.8254
Epoch 4/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0190 - accuracy: 0.9980 - val_loss: 0.5310 - val_accuracy: 0.8260
Epoch 5/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0048 - accuracy: 0.9996 - val_loss: 0.5873 - val_accuracy: 0.8276
Epoch 6/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.6386 - val_accuracy: 0.8270
Epoch 7/10
782/782 [==============================] - 1s 2ms/step - loss: 8.0511e-04 - accuracy: 1.0000 - val_loss: 0.6700 - val_accuracy: 0.8294
Ep

In [12]:
# 데이터 그래픽화
e = model.layers[0] # 임베딩 레이어(tf.keras.layers.Embedding)의 출력값. 인덱스가 0인데 임베딩 레이어가 나오는 이유는 모델 만들 때 얘를 제일 먼저 넣었기 때문. 
weights = e.get_weights()[0] # 임베딩 레이어의 가중치
print(weights.shape) # shape: (vocab_size, embedding_dim) 단어가 10000개고 임베딩 차원이 16차원임

(10000, 16)


In [13]:
# 파일 제작
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8') # 16차원 벡터값 저장
out_m = io.open('meta.tsv', 'w', encoding='utf-8') # 벡터에 해당하는 단어 저장
for word_num in range(1, vocab_size): 
  word = reverse_word_index[word_num] # 앞서 플립을 통해 [단어 : 토큰] -> [토큰 : 단어]로 만들었다. 그래서 토큰(숫자)가 인덱스 넘버인 배열처럼 사용할 수 있다
  embeddings = weights[word_num] # 임베딩 가중치
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [14]:
# 제작한 tsv파일 다운로드. 얘들을 임베딩 프로젝터에서 load하면 임베딩 그래프를 볼 수 있다. 단어를 검색하면 그 단어의 위치가 나오고 비슷한 방향을 지닌 단어들이 주변에 나타난다. 
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [15]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
